# Generate Data for Analysis of Iris Visibility

This notebook generates data for analysis of glint visibility.
Generation of 100 combined user-device samples x two eyes x 20 gaze directions,
i.e., 4,000 individual samples, takes about 10-15 min on a beefy machine.

In [ ]:
%matplotlib widget

import datetime
import seet
import os
import pickle
import sys
import torch
import seet.core as core
import pandas

sys.path.append("..")
import utils

dropdown, text = utils.get_experiment_info()

## Data Generation

This takes 10-15 min on a beefy machine.

In [ ]:
def generate_data_for_iris_analysis(scene_sampler, num_angles=10, num_radii=10):
    """generate_data_for_iris_analysis.

    Generate data with area of visible iris in mm, and area of projected
    iris in camera in pixels, as well as percentage of iris that is
    visible. The data is for nominal gaze direction only, as this is a very
    expensive computation.

    Args:
        num_angles (int, optional): number of angles to be sampled around
        the iris. Defaults to 10.

        num_radii (int, optional): number of radii to be sample for each
        angle. Defaults to 10.

    Returns:
        pandas.DataFrame: pandas data frame with columns corresponding to
        scene subsystem, percentage of iris visible, area of visible iris
        in mm^2, and area of the projection of the visible iris in the
        camera, in pixels^2.
    """

    header = [
        "Subsystem",
        "Percentage visible",
        "Area in iris [mm^2]",
        "Area in image [pix^2]"
    ]

    # Parameters to compute area element
    d_theta = torch.tensor(2 * torch.pi / num_angles, requires_grad=True)

    num_subsystems = len(scene_sampler.scene.device.subsystems)
    data = []
    for et_scene in scene_sampler.generate_samples():
        for subsystem_index in range(num_subsystems):
            camera = et_scene.device.subsystems[subsystem_index].cameras[0]
            eye = et_scene.user.eyes[subsystem_index]
            cornea = eye.cornea
            limbus = eye.limbus

            ###############################################################
            # Sample iris in polar coordinates.
            optical_center_inCornea = \
                camera.get_optical_center_inOther(cornea)
            transform_toCornea_fromLimbus = \
                limbus.get_transform_toOther_fromSelf(cornea)

            d_r = limbus.radius / num_radii
            # TODO: This is ugly! We need to create a function T0() that
            # generates the detached value of the zero tensor.
            area_total_mm = core.T0.clone().detach()
            area_mm = core.T0.clone().detach()
            area_pix = core.T0.clone().detach()
            for i in range(num_angles):
                theta = i * d_theta
                for j in range(num_radii):
                    r = j * d_r
                    x = r * torch.cos(theta)
                    y = r * torch.sin(theta)
                    point_in2DIris = torch.hstack((x, y))

                    point_in3DIris = \
                        torch.hstack((point_in2DIris, core.T0))
                    iris_point_inCornea = \
                        transform_toCornea_fromLimbus.transform(
                            point_in3DIris
                        )

                    # Make sure point is inside cornea.
                    if cornea.compute_algebraic_distance_inEllipsoid(
                        iris_point_inCornea
                    ) >= 0:
                        continue

                    d_x_d_y = r * d_r * d_theta
                    area_total_mm += d_x_d_y

                    refraction_point_inCornea = \
                        cornea.compute_refraction_point_inEllipsoid(
                            optical_center_inCornea,
                            iris_point_inCornea,
                            eta_at_destination=cornea.refractive_index
                        )
                    if refraction_point_inCornea is None:
                        continue

                    # Check occlusion by device occluder, if one is
                    # present.
                    unit_list_refraction_point_inCornea = \
                        et_scene.device.subsystems[
                            subsystem_index
                        ].apply_occluder_inOther(
                            cornea,
                            [refraction_point_inCornea, ],  # list
                            reference_point_inOther=optical_center_inCornea
                        )
                    # Input is list of points, and so is output.
                    refraction_point_inCornea = \
                        unit_list_refraction_point_inCornea[0]

                    if refraction_point_inCornea is not None:
                        area_mm += d_x_d_y

                        refraction_point_inPixels = \
                            camera.project_toPixels_fromOther(
                                refraction_point_inCornea, cornea
                            )

                        d_inPixels_d_in2DIris = \
                            core.compute_auto_jacobian_from_tensors(
                                refraction_point_inPixels, point_in2DIris
                            )

                        area_pix += \
                            d_x_d_y * \
                            torch.abs(
                                torch.linalg.det(d_inPixels_d_in2DIris)
                            )

            percentage = (100 * area_mm / area_total_mm).item()
            data = \
                data + \
                [
                    [
                        subsystem_index,
                        percentage,
                        area_mm.item(),
                        area_pix.item()
                    ]
                ]

    return pandas.DataFrame(data, columns=header)

In [ ]:
scene_file_name, \
    sampler_file_name = \
    utils.get_configuration_files(dropdown.value)

print("Scene generated using " + scene_file_name + " configuration file.")
et_scene = \
    seet.scene.SceneModel(
        parameter_file_name=scene_file_name, requires_grad=True
    )

print("Sampling parameters from " + sampler_file_name + " configuration file.")
scene_sampler = seet.sampler.SceneSampler(
    et_scene, num_samples=100, parameter_file_name=sampler_file_name)

# df = scene_sampler.generate_data_for_iris_analysis()
df = generate_data_for_iris_analysis(scene_sampler)

now = datetime.datetime.now()
prefix = "iris - " + now.strftime("%Y-%m-%d @ %H-%M-%S.%f")
results_path = text.value
os.makedirs(results_path, exist_ok=True)
path_prefix = os.path.join(results_path, prefix)

df_name = path_prefix + " data_frame.pkl"
with open(df_name, 'wb') as file_stream:
    pickle.dump(df, file_stream)

df